# Imports 

In [3]:
"""

pip install scipy
pip install h5py
pip install matplotlib
pip install opencv-python
pip install seaborn
pip install matplotlib
pip install ipywidgets
pip install matplotlib install ipywidgets
pip install opencv-pythonjupyter nbextension enable --py widgetsnbextension
pip install ipython
pip install WordCloud
pip install geopy
pip install scipy
pip install plotly
pip install nbformat --upgrade
pip install folium
pip install geopandas
pip install scikit-learn   
pip install cairosvg
pip install lxm



pip install selenium
pip install pandas
pip install python-dotenv
pip install openai
pip install beautifulsoup4
pip install numpy
pip install tqdm
pip install seaborn
pip install matplotlib
pip install scipy
pip install geopy
pip install plotly



"""

'\n\npip install scipy\npip install h5py\npip install matplotlib\npip install opencv-python\npip install seaborn\npip install matplotlib\npip install ipywidgets\npip install matplotlib install ipywidgets\npip install opencv-pythonjupyter nbextension enable --py widgetsnbextension\npip install ipython\npip install WordCloud\npip install geopy\npip install scipy\npip install plotly\npip install nbformat --upgrade\npip install folium\npip install geopandas\npip install scikit-learn   \npip install cairosvg\npip install lxm\n\n\n\npip install selenium\npip install pandas\npip install python-dotenv\npip install openai\npip install beautifulsoup4\npip install numpy\npip install tqdm\npip install seaborn\npip install matplotlib\npip install scipy\npip install geopy\npip install plotly\n\n\n\n'

In [4]:
gpt_instructions = """
for the below text using using your LLM ability (not python code) extract data from the text. get the following and save all as strings, if they don't exist for a specific job entry, use an empty string (i.e., ""). 
there will be exactly 20 of each. 

job_title 
company_name 
location

format the output as a python single dictionary for all the jobs. for example all the job titles will use key "job_title" and be a list of the jobs

I need to have have a standard output and use it directly for my analysis. process every single job 

Construct the dictionary: Organize all the extracted information into a dictionary format where the key represents the type of information and the value is a list of all the extracted information of that type in the order of appearance.

Formatting the instructions: Always make sure to start the code block with a comment '##BEGIN##', fill in the data accordingly, and end it with '##END##'. All columns of data should align with each other.

    For example:


    ##BEGIN##
job_data = {
    "job_title": [...],
    "company_name": [...],
    "location": [...]
}
    ##END##

    Please make sure to follow these instructions to achieve consistent results.

    here is the text:



    
"""

In [5]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from dotenv import load_dotenv
import os
import openai

from bs4 import BeautifulSoup
import re
import time
import pickle
import numpy as np 
import pandas as pd

from scrapifurs import utils
from scrapifurs.GPTinstructions import GPTinstructions

from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.cluster.hierarchy import linkage, leaves_list

from geopy.geocoders import Nominatim
from geopy.distance import geodesic


import plotly.express as px

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime





In [6]:
def get_lat_lon(locations):
    geolocator = Nominatim(user_agent="example.lastname@gmail.com")
    lat_lon_data = {}
    for location in tqdm(locations):
        location_data = geolocator.geocode(location)
        if location_data is not None:
            lat_lon_data[location] = (location_data.latitude, location_data.longitude)
        else:
            lat_lon_data[location] = (None, None)
    return lat_lon_data

def calculate_distances(lat_lon_data, target_loc_str):
    target_location = get_lat_lon([target_loc_str])[target_loc_str]
    if target_location[0] is None:
        raise ValueError("No latitude and longitude found for the provided location.")
    distances = {}
    for location, loc_lat_lon in lat_lon_data.items():
        if loc_lat_lon[0] is not None:
            distance = geodesic(target_location, loc_lat_lon).miles
            distances[location] = distance
        else:
            distances[location] = None
    return distances





def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    union_length = len(set1.union(set2))
    if union_length == 0:
        return np.nan
    return len(set1.intersection(set2)) / union_length


class GPT_StringDataCleaner:
    def __init__(self, original_df, key_name, instructions, overwrite_cleaned_key = True, model="gpt-4"):
        self.original_df = original_df
        self.key_name = key_name
        self.instructions = instructions
        self.model = model
        self.clean_key = key_name + '_CLEANED_BY_GPT'
        self.overwrite_cleaned_key = overwrite_cleaned_key

    def send_to_gpt(self):
        self.unique_names = sorted(self.original_df[self.key_name].unique())

        messages = [{"role": "system", "content": self.instructions}]
        messages.append({"role": "user", "content": str(self.unique_names)})
        chat = openai.ChatCompletion.create(model=self.model, messages=messages)
        self.reply = chat.choices[0].message.content
    def test_lengths(self):
        print(f" len input is {len(self.unique_names)}, but len output is {len(cleaned_data)}")
    def process_data(self):
        cleaned_data =  eval(self.reply)
        
        if len(self.unique_names) != len(cleaned_data):
            print(f" len input is {len(self.unique_names)}, but len output is {len(cleaned_data)}")
            raise ValueError("Input and output lists must have the same length")

        temp_df = pd.DataFrame({self.key_name: self.unique_names, self.clean_key: cleaned_data})
        # delete cleaned key data if we want to overwrite
        if self.clean_key in self.original_df.columns and self.overwrite_cleaned_key:
            self.original_df = self.original_df.drop(self.clean_key, axis=1)
        if self.clean_key not in self.original_df.columns:
            merged_df = pd.merge(self.original_df, temp_df[[self.clean_key, self.key_name]],
                                 on=self.key_name, how='left')
            self.original_df[self.clean_key] = merged_df[self.clean_key]
            merged_df = None

        return self.original_df

In [7]:

def update_master_df(job_data_all, job_data):
    # Reverse the order of job_data to check oldest entries first
    job_data_reversed = job_data.iloc[::-1]
    
    # List to hold new entries
    new_entries = []
    
    for i, row in job_data_reversed.iterrows():
        # Check if row exists in job_data_all
        is_exist = job_data_all[(job_data_all['job_title'] == row['job_title']) & 
                                (job_data_all['company_name'] == row['company_name']) &
                                (job_data_all['location'] == row['location'])].shape[0]
        
        # If row does not exist in job_data_all, add it to new_entries
        if is_exist == 0:
            print('NEWWWWWW')
            print(row)
            new_entries.append(row)
    
    # Convert new_entries to DataFrame and concatenate it with job_data_all
    if new_entries:
        new_entries_df = pd.DataFrame(new_entries)
        job_data_all = pd.concat([new_entries_df, job_data_all], ignore_index=True)
    
    return job_data_all


def get_job_blocks(gpt_text, job_data):
    job_blocks = []
    combined_titles = "|".join([re.escape(title) for title in job_data['job_title']])
    combined_companies = "|".join([re.escape(company) for company in job_data['company_name']])
    pattern = f"({combined_titles}).*?({combined_companies}).*?(?=({combined_titles}).*?({combined_companies})|$)"
    
    for match in re.finditer(pattern, gpt_text, re.DOTALL):
        job_blocks.append(match.group(0))

    return job_blocks

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def go_to_next_page(driver):
    try:
        # Step 1: Find the pagination element
        pagination = driver.find_element(By.CSS_SELECTOR, 'ul.artdeco-pagination__pages')
    except NoSuchElementException:
        print("Pagination not found")
        return

    try:
        # Step 2: Find the current active page
        current_page_elem = pagination.find_element(By.CSS_SELECTOR, 'li.active')
        current_page = int(current_page_elem.text)
    except NoSuchElementException:
        print("Current page not found")
        return

    # Step 3: Find the next page
    next_page = current_page + 1
    next_page_selector = f'li[data-test-pagination-page-btn="{next_page}"]'

    try:
        next_page_elem = pagination.find_element(By.CSS_SELECTOR, next_page_selector)
    except NoSuchElementException:
        print("Next page not found")
        return

    # Step 4: Click the next page
    next_page_elem.click()


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

def update_dataframe(df_main, df_new, keys=None):
    if keys is None:
        keys = ['job_ids', 'job_title', 'company_name', 'Location', 'pay']

    # Check if types are the same for each key
    mismatched_keys = []
    for key in keys:
        type_main = df_main[key].dtype
        type_new = df_new[key].dtype
        if type_main != type_new:
            mismatched_keys.append((key, type_main, type_new))

    if mismatched_keys:
        print("Type mismatch for these keys:")
        for key, type_main, type_new in mismatched_keys:
            print(f"Key: {key}, Type in df_main: {type_main}, Type in df_new: {type_new}")
        raise TypeError("Type mismatch detected.")
        
    for _, new_row in df_new.iterrows():
        mask = df_main[keys].eq(new_row[keys]).all(axis=1)
        if not any(mask):
            df_main = pd.concat([df_main, pd.DataFrame([new_row])], ignore_index=True)
            os.system('say "youve got jobs"')  # macOS
            print('\nNNNNNEEWWWWWWW')
            print(new_row['job_link'])
            print(new_row)

    return df_main


# def update_dataframe(df_main, df_new, keys=None):
#     if keys is None:
#         # keys = df_main.columns.tolist()
#         keys = ['job_ids', 'job_title', 'company_name', 'Location', 'pay']
        
        
#     for i, new_row in df_new.iterrows():
#         if not any(df_main[keys].eq(new_row[keys]).all(axis=1)):
            
#             df_main = pd.concat([df_main, pd.DataFrame([new_row])], ignore_index=True)
#             print('\nNNNNNEEWWWWWWW')
#             print('\n')
#             print(new_row['job_link'])
#             print(new_row)
#             asdfasdfasdf
#     return df_main

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def get_job_details(driver):
    job_dict = {
        "job_ids": [],
        "is_promoted": [],
        "job_title": [],
        "company_name": [],
        "Location": [],
        "pay": [],
        "job_link": [],
        "time_added": [],
    }
    
    jobs = driver.find_elements(By.CSS_SELECTOR, '[data-occludable-job-id]')
    
    for job in jobs:
        try:
            job_id = job.get_attribute("data-occludable-job-id")
        except NoSuchElementException:
            job_id = -1
        
        try:
            promoted_elements = job.find_elements(By.CSS_SELECTOR, '.job-card-container__footer-item')
            is_promoted = "Promoted" in [e.text for e in promoted_elements]
        except NoSuchElementException:
            is_promoted = False

        try:
            job_title = job.find_element(By.CSS_SELECTOR, '.job-card-list__title').text
        except NoSuchElementException:
            job_title = ""

        try:
            company_name = job.find_element(By.CSS_SELECTOR, '.job-card-container__primary-description').text
        except NoSuchElementException:
            company_name = ""

        try:
            Location = job.find_element(By.CSS_SELECTOR, '.job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            Location = ""

        try:
            pay = job.find_element(By.CSS_SELECTOR, '.mt1 .job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            pay = ""
            
        try:
            job_link = job.find_element(By.CSS_SELECTOR, "a.job-card-container__link").get_attribute("href")
        except NoSuchElementException:
            job_link = ""
            
        

        if job_title != "" and company_name != "":
            job_dict["job_ids"].append(job_id)
            job_dict["is_promoted"].append(is_promoted)
            job_dict["job_title"].append(job_title)
            job_dict["company_name"].append(company_name)
            job_dict["Location"].append(Location)
            job_dict["pay"].append(pay)
            job_dict["job_link"].append(job_link)
            
            current_time = datetime.now()
            formatted_time = current_time.strftime('%Y%m%d%H%M%S')
            formatted_time_as_int = int(formatted_time)
            job_dict["time_added"].append(formatted_time_as_int)
            
        
    return job_dict


### INIT API key,  intrucitons for GPT which can be saved as text files in the correct data/instrucitons dir of the package

In [9]:

# # setup API key for chatGPT 
# load_dotenv()  # take environment variables from .env.
# os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY")
# openai.api_key = os.environ["OPENAI_API_KEY"]




# Info_dict to set up all used variables

In [10]:
# setup basic variable as dict 
info_dict = {'init_url':'https://www.linkedin.com/',
             'save_password_dir':'/Users/phil/Dropbox/GITHUB/DATA/scrapifurs/saved_cookies/',
             'start_url':'https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=fRq'}
info_dict['full_cookies_save_path'] = info_dict['save_password_dir']+os.sep+"linkedin_cookies.pkl"


# Scraping step

### create rules to trim text based on key workds and search characteristics
this willl help reduce the amount of data we pass to GPT API and reduce costs 

In [11]:
text_finder = utils.StringSectionExtractor()
text_finder.add_start_rule('\d+ results', True)
text_finder.add_start_rule('All filters', False)
text_finder.add_start_rule('Jump to active job details', False)


text_finder.add_end_rule('Page \d+ of \d+', True)
text_finder.add_end_rule("Are you finding", False)

### INIT: chrome browser, login, save cookies (future login)

In [16]:


#init chrome 
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=chrome_options)

driver.get(info_dict['init_url'])
time.sleep(1)
driver.get(info_dict['init_url'])
time.sleep(2)


# Load cookies if they exist
try:
    cookies = pickle.load(open(info_dict['full_cookies_save_path'], "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.refresh()
    assert(not not cookies)# if empty try a different method
except:
    print("No cookies found. Manual login required.")
    # If not logged in
    input('Please login and press Enter to continue...')
    pickle.dump(driver.get_cookies(), open(info_dict['full_cookies_save_path'], "wb")) # save cookies after login
    

time.sleep(4)

In [ ]:
# test scraping full job page for saving data for all jobs I apllied for 

In [19]:
x = 'https://www.linkedin.com/jobs/view/3743737484/?eBP=JOB_SEARCH_ORGANIC&refId=wIC2skgivSr27kABYstWjg%3D%3D&trackingId=XVrPkXaydKf4xGrJyLPUOw%3D%3D'
info_dict['start_url'] = x
driver.get(info_dict['start_url'])



In [25]:
from selenium.common.exceptions import NoSuchElementException

def get_job_detail(driver):
    job_detail_dict = {
        "job_title": "",
        "company_name": "",
        # ... add other fields as necessary
    }

    # Scrape the job title
    try:
        job_detail_dict["job_title"] = driver.find_element(By.CSS_SELECTOR, 'h1.t-24.t-bold.job-details-jobs-unified-top-card__job-title').text.strip()
    except NoSuchElementException:
        job_detail_dict["job_title"] = ""

    # Scrape the company name
    try:
        job_detail_dict["company_name"] = driver.find_element(By.CSS_SELECTOR, 'a.app-aware-link').text.strip()
    except NoSuchElementException:
        job_detail_dict["company_name"] = ""

    # ... similarly add extraction logic for any other information you need

    return job_detail_dict

get_job_detail(driver)

{'job_title': 'Data Scientist', 'company_name': ''}

In [60]:
# Function to safely fetch elements
def safe_extract(selector):
    try:
        return driver.find_element(By.CSS_SELECTOR, selector).text
    except Exception as e:
        return f"Error: {e}"

# Extract contents with updated selectors
job_title = safe_extract('h1')  # This one works
company_info = safe_extract('.job-details-jobs-unified-top-card__subtitle span:nth-child(1)')  # Adjusting selector
job_description = safe_extract('.description')  # Using a more common class name for job descriptions

print(job_title)
# print(company_info)
# print(job_description)


Data Scientist


In [61]:

# Define a function to safely extract elements
def safe_extract(element, css_selector, attribute=None):
    try:
        if attribute:
            return element.find_element(By.CSS_SELECTOR, css_selector).get_attribute(attribute)
        return element.find_element(By.CSS_SELECTOR, css_selector).text
    except NoSuchElementException:
        return None

# Extracting data from the provided HTML
job_title = safe_extract(driver, '.job-details-jobs-unified-top-card__job-title')
company_name = safe_extract(driver, '.job-details-jobs-unified-top-card__primary-description a')
company_logo_url = safe_extract(driver, '.ivm-view-attr__img--centered', 'src')
job_location = safe_extract(driver, '.job-details-jobs-unified-top-card__primary-description span:nth-child(2)')
time_since_posted = safe_extract(driver, '.tvm__text--neutral span')
num_applicants = safe_extract(driver, '.tvm__text--neutral:last-child')
salary_range = safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight span span:first-child')
job_type = safe_extract(driver, '.ui-label.ui-label--accent-3:nth-child(1) span:first-child')
job_level = safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight-view-model-secondary:nth-child(3)')
company_info = safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight:nth-child(2) span')

job_description_element = ".jobs-description-content__text"
description = safe_extract(driver, job_description_element)




print(f"Job Title: {job_title}")
print(f"Company Name: {company_name}")
print(f"Company Logo URL: {company_logo_url}")
print(f"Job Location: {job_location}")
print(f"Time Since Posted: {time_since_posted}")
print(f"Number of Applicants: {num_applicants}")
print(f"Salary Range: {salary_range}")
print(f"Job Type: {job_type}")
print(f"Job Level: {job_level}")
print(f"Company Info: {company_info}")
print(description)

Job Title: Data Scientist
Company Name: Expa
Company Logo URL: https://media.licdn.com/dms/image/C4E0BAQGYjwhcNh5EVg/company-logo_100_100/0/1613080862386/expa_logo?e=1707350400&v=beta&t=-CR39e7rJogy2WMQ3q9z4H8BCPU7GUMjaJYG0ixayoM
Job Location:  
Time Since Posted: 1 week ago
Number of Applicants: Over 100 applicants
Salary Range: $100,000/yr - $120,000/yr
Job Type: Hybrid
Job Level: Full-time
Matches your job preferences, job type is Full-time.
Company Info: 11-50 employees · Technology, Information and Internet


In [63]:
job_description_element = ".jobs-description-content__text"
description = safe_extract(driver, job_description_element)

# print(description)

In [67]:
from selenium.webdriver.common.by import By

# Use the find_element method with By.CSS_SELECTOR to get the location
location_div = driver.find_element(By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__primary-description div")
location_text = location_div.text

# Since the location is preceded by a `·`, we'll split by it and take the second item
job_location = location_text.split('·')[1].strip()
print(job_location)  # Outputs: Los Angeles, CA


Los Angeles, CA  1 week ago


In [68]:
from selenium.webdriver.common.by import By

# Use the find_element method with By.CSS_SELECTOR to get the location
location_div = driver.find_element(By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__primary-description div")
location_parts = location_div.text.split('·')

# Assuming the location is always the second item after splitting by `·`, but stripping any additional info after city and state/country
potential_location = location_parts[1].strip().split()

# Checking for common separators to extract the primary location (like ",", "-")
if "," in potential_location[1]:
    job_location = ' '.join(potential_location[:2])
else:
    job_location = ' '.join(potential_location[:1])

print(job_location)


Los Angeles,


In [84]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def get_apply_link(driver):
    try:
        # 1. Open the link in a new tab using JavaScript.
        apply_button = driver.find_element(By.CSS_SELECTOR, ".jobs-apply-button.artdeco-button")
        driver.execute_script("window.open(arguments[0].click(), '_blank');", apply_button)
        # 2. Switch to the new tab.
        driver.switch_to.window(driver.window_handles[1])
        # 3. Copy the URL.
        apply_link = driver.current_url
        # 4. Close the new tab.
        driver.close()
        # 5. Switch back to the original tab.
        driver.switch_to.window(driver.window_handles[0])
        return apply_link
    except Exception as e:
        print(f"Error: {e}")
        return None


def get_location(element):
    location_div = element.find_element(By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__primary-description div")
    location_parts = location_div.text.split('·')
    potential_location = location_parts[1].strip().split()
    if "," in potential_location[1]:
        return ' '.join(potential_location[:2])
    else:
        return ' '.join(potential_location[:1])

# Define a function to safely extract elements
def safe_extract(element, css_selector, attribute=None):
    try:
        if attribute:
            return element.find_element(By.CSS_SELECTOR, css_selector).get_attribute(attribute)
        return element.find_element(By.CSS_SELECTOR, css_selector).text
    except NoSuchElementException:
        return None



# Extracting data from the provided HTML
data = {
    "linked_in_link": driver.current_url,
    "job_title": safe_extract(driver, '.job-details-jobs-unified-top-card__job-title'),
    "company_name": safe_extract(driver, '.job-details-jobs-unified-top-card__primary-description a'),
    "apply_link": get_apply_link(driver),
    "job_location": get_location(driver),
    "time_since_posted": safe_extract(driver, '.tvm__text--neutral span'),
    "num_applicants": safe_extract(driver, '.tvm__text--neutral:last-child'),
    "salary_range": safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight span span:first-child'),
    "job_type": safe_extract(driver, '.ui-label.ui-label--accent-3:nth-child(1) span:first-child'),
    "job_level": safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight-view-model-secondary:nth-child(3)'),
    "company_info": safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight:nth-child(2) span'),
    "description": safe_extract(driver, ".jobs-description-content__text")
}
data = {
    key: [value] for key, value in data.items()
}

df = pd.DataFrame(data)
df


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=118.0.5993.117)
Stacktrace:
0   chromedriver                        0x0000000101278510 chromedriver + 4310288
1   chromedriver                        0x00000001012704bc chromedriver + 4277436
2   chromedriver                        0x0000000100ea3b6c chromedriver + 293740
3   chromedriver                        0x0000000100e8b39c chromedriver + 193436
4   chromedriver                        0x0000000100e8b2a4 chromedriver + 193188
5   chromedriver                        0x0000000100f2370c chromedriver + 816908
6   chromedriver                        0x0000000100edcfd0 chromedriver + 528336
7   chromedriver                        0x0000000100edde7c chromedriver + 532092
8   chromedriver                        0x000000010123e834 chromedriver + 4073524
9   chromedriver                        0x00000001012427fc chrome

In [81]:
# # Locate the "Apply" button and click it
# apply_button = driver.find_element(By.CSS_SELECTOR, ".jobs-apply-button.artdeco-button")
# apply_button.click()
# time.sleep(4)
# # After clicking, get the current URL
# apply_link2 = driver.current_url
# apply_link2


In [83]:
job_id_match = re.search(r'/view/(\d+)/', data["linked_in_link"])
job_id = job_id_match.group(1) if job_id_match else None

 data["linked_in_link"]

IndentationError: unexpected indent (2733806538.py, line 4)

In [ ]:
"""
get linked in link 
linked in job number use this as the index 


"""

In [79]:
import re

def get_job_id(element, css_selector):
    content = safe_extract(element, css_selector)
    
    # Use regex to find the job ID number from the content
    job_id_match = re.search(r'/voyager/api/jobs/jobPostings/(\d+)', content)
    if job_id_match:
        return job_id_match.group(1)
    return None

# Use the function
job_id = get_job_id(driver, '#datalet-bpr-guid-3188948')


TypeError: expected string or bytes-like object, got 'NoneType'

In [49]:
def get_element_data(element):
    data = {
        "tag": element.tag_name,
        "class": element.get_attribute('class'),
        "id": element.get_attribute('id'),
        "text": element.text.strip(),
        "children": []
    }

    for child in element.find_elements_by_xpath("./*"):
        if "job" in child.get_attribute('class').lower() or \
           "job" in child.get_attribute('id').lower() or \
           "job" in child.get_attribute('name').lower():
            child_data = get_element_data(child)
            data["children"].append(child_data)
            # Removing child's text from parent to avoid duplication
            data["text"] = data["text"].replace(child_data["text"], '').strip()

    return data

elements_with_job = WebDriverWait(driver, 1).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        "[class*='job'], [id*='job'], [name*='job']"))
)

all_elements_data = []

for element in elements_with_job:
    all_elements_data.append(get_element_data(element))

print(all_elements_data)


AttributeError: 'WebElement' object has no attribute 'find_elements_by_xpath'

In [ ]:

def get_job_details(driver):
    job_dict = {
        "job_ids": [],
        "is_promoted": [],
        "job_title": [],
        "company_name": [],
        "Location": [],
        "pay": [],
        "job_link": [],
        "time_added": [],
    }
    
    jobs = driver.find_elements(By.CSS_SELECTOR, '[data-occludable-job-id]')
    
    for job in jobs:
        try:
            job_id = job.get_attribute("data-occludable-job-id")
        except NoSuchElementException:
            job_id = -1
        
        try:
            promoted_elements = job.find_elements(By.CSS_SELECTOR, '.job-card-container__footer-item')
            is_promoted = "Promoted" in [e.text for e in promoted_elements]
        except NoSuchElementException:
            is_promoted = False

        try:
            job_title = job.find_element(By.CSS_SELECTOR, '.job-card-list__title').text
        except NoSuchElementException:
            job_title = ""

        try:
            company_name = job.find_element(By.CSS_SELECTOR, '.job-card-container__primary-description').text
        except NoSuchElementException:
            company_name = ""

        try:
            Location = job.find_element(By.CSS_SELECTOR, '.job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            Location = ""

        try:
            pay = job.find_element(By.CSS_SELECTOR, '.mt1 .job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            pay = ""
            
        try:
            job_link = job.find_element(By.CSS_SELECTOR, "a.job-card-container__link").get_attribute("href")
        except NoSuchElementException:
            job_link = ""
            
        

        if job_title != "" and company_name != "":
            job_dict["job_ids"].append(job_id)
            job_dict["is_promoted"].append(is_promoted)
            job_dict["job_title"].append(job_title)
            job_dict["company_name"].append(company_name)
            job_dict["Location"].append(Location)
            job_dict["pay"].append(pay)
            job_dict["job_link"].append(job_link)
            
            current_time = datetime.now()
            formatted_time = current_time.strftime('%Y%m%d%H%M%S')
            formatted_time_as_int = int(formatted_time)
            job_dict["time_added"].append(formatted_time_as_int)
            
        
    return job_dict

#### navigate to the first page we want to scrape

In [28]:

# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3481251151&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C288%2C2463%2C25584&f_WT=1&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3682851114&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currefntJobId=3642657953&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_TPR=r86400&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3737611604&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_TPR=r604800&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# # x = 'https://www.linkedin.com/jobs/search/?currentJobId=3729935051&distance=25&f_PP=102277331%2C102448103%2C106233382%2C103918656%2C100075706%2C102250832%2C103575230&f_TPR=r86400&f_WT=1%2C3%2C2&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3741083392&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_TPR=r86400&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# info_dict['start_url'] = x
# driver.get(info_dict['start_url'])
# time.sleep(5)


In [29]:
# df_main = pd.read_csv('/Users/phil/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/DS_mega_data_V1.csv')
# # df_main['job_ids'] = df_main['job_ids'].astype(int)
# # df_main.to_csv('/Users/phil/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/DS_mega_data_V1.csv', index=False)



In [27]:
# # job_data = get_job_details(driver)
# # df_main = pd.DataFrame(job_data)
# # df_main

# n_pages_to_scrape = 5
# wait_sec_each_page = 10
# update_every_n_secs = 60*6





# while True:
#     for n_page in range(n_pages_to_scrape):
#         if n_page ==0:
#             # go back to the first page
#             driver.get(info_dict['start_url'])
#         else:
#             go_to_next_page(driver)
#         time.sleep(wait_sec_each_page)
#         # update data
#         job_data = get_job_details(driver)
#         job_data = pd.DataFrame(job_data)
#         job_data['job_ids'] = job_data['job_ids'].astype('int')
#         df_main = update_dataframe(df_main, job_data, ['job_ids'])
    
#     # save CSV 
#     df_main.to_csv('/Users/phil/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/DS_mega_data_V1.csv', index=False)
    
#     print('_____________________________________________\n\n____________________')
#     time.sleep(update_every_n_secs)
